In [1]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
root_dir = "/content/drive/MyDrive/Real_Estate_Classification"
path = os.chdir(root_dir)
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1nLL5DNOyzlfE3_Obr8tg83RSxo_WDfRS/Real_Estate_Classification'

## Import library

In [3]:
import os
import shutil
import pandas as pd
import random
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

## Data preprocessing

In [4]:
train_df = pd.read_csv("REMIS_Image_Dataset/train_re.csv")
train_df['labels'] = [train_df['path'].str.split('\\')[i][1] for i in range(len(train_df['path']))]
train_df['path'] = train_df['path'].str.replace('Dataset', 'REMIS_Image_Dataset')
train_df['path'] = train_df['path'].str.replace('\\', '/')
train_df['name'] = ['Train' + str(i) + '.jpg' for i in range(len(train_df['path'])) ]
train_df

,path,label,labels,name
0,REMIS_Image_Dataset/investor/rever_phu-my-thua...,"[1, 0, 0, 0]",investor,Train0.jpg
1,REMIS_Image_Dataset/others/10215375959772007.jpeg,"[0, 1, 0, 0]",others,Train1.jpg
2,REMIS_Image_Dataset/project/duanbatdongsan_pj5...,"[0, 0, 0, 1]",project,Train2.jpg
3,REMIS_Image_Dataset/post/123nhadat_4321333_0.jpg,"[0, 0, 1, 0]",post,Train3.jpg
4,REMIS_Image_Dataset/investor/rever_keppel-land...,"[1, 0, 0, 0]",investor,Train4.jpg
...,...,...,...,...
895,REMIS_Image_Dataset/investor/rever_tan-thanh-d...,"[1, 0, 0, 0]",investor,Train895.jpg
896,REMIS_Image_Dataset/others/10153721284885397.jpeg,"[0, 1, 0, 0]",others,Train896.jpg
897,REMIS_Image_Dataset/others/3744275319541.jpeg,"[0, 1, 0, 0]",others,Train897.jpg
898,REMIS_Image_Dataset/others/1012645588840463.jpeg,"[0, 1, 0, 0]",others,Train898.jpg


In [5]:
test_df = pd.read_csv("REMIS_Image_Dataset/test_re.csv")
test_df['labels'] = [test_df['path'].str.split('\\')[i][1] for i in range(len(test_df['path']))]
test_df['path'] = test_df['path'].str.replace('Dataset', 'REMIS_Image_Dataset')
test_df['path'] = test_df['path'].str.replace('\\', '/')
test_df['name'] = ['Test' + str(i) + '.jpg' for i in range(len(test_df['path'])) ]
test_df

,path,label,labels,name
0,REMIS_Image_Dataset/others/837645183007172.jpeg,"[0, 1, 0, 0]",others,Test0.jpg
1,REMIS_Image_Dataset/post/123nhadat_4321799_4.jpg,"[0, 0, 1, 0]",post,Test1.jpg
2,REMIS_Image_Dataset/others/10208474191744749.jpeg,"[0, 1, 0, 0]",others,Test2.jpg
3,REMIS_Image_Dataset/investor/rever_bat-dong-sa...,"[1, 0, 0, 0]",investor,Test3.jpg
4,REMIS_Image_Dataset/project/duanbatdongsan_pj5...,"[0, 0, 0, 1]",project,Test4.jpg
...,...,...,...,...
255,REMIS_Image_Dataset/investor/rever_gia-tue_0.jpg,"[1, 0, 0, 0]",investor,Test255.jpg
256,REMIS_Image_Dataset/post/123nhadat_4310483_0.jpg,"[0, 0, 1, 0]",post,Test256.jpg
257,REMIS_Image_Dataset/others/10200930046625836.jpeg,"[0, 1, 0, 0]",others,Test257.jpg
258,REMIS_Image_Dataset/others/1047558682015820.jpeg,"[0, 1, 0, 0]",others,Test258.jpg


## Create training and testing dataset

In [ ]:
try:
  os.mkdir('data')
  os.mkdir('data/training')
  os.mkdir('data/testing')
except:
  pass

In [ ]:
for i, path in enumerate(train_df['path']):
  try:
    shutil.copy(path, "data/training/Train"+str(i)+".jpg")
  except:
    pass

for i, path in enumerate(test_df['path']):
  try:
    shutil.copy(path, "data/testing/Test"+str(i)+".jpg")
  except:
    pass

In [6]:
## Initalize Image Data Generator with Augmentation
train_data_generator = ImageDataGenerator(rescale=1./255,
        rotation_range=180,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        brightness_range = (0.9,1.1),
        fill_mode='nearest')


## Recreate datasets from dataframe
train_data_multi = train_data_generator.flow_from_dataframe(dataframe=train_df,
                                                    directory="data/training",
                                                    x_col="name",
                                                    y_col= "labels",
                                                    target_size=(224, 224),
                                                    class_mode='categorical',
                                                    batch_size=16,
                                                    shuffle=True,
                                                    seed=42)

test_data_generator = ImageDataGenerator(rescale=1/255.)

test_data_multi = train_data_generator.flow_from_dataframe(dataframe=test_df,
                                                    directory="data/testing",
                                                    x_col="name",
                                                    y_col= "labels",
                                                    target_size=(224, 224),
                                                    class_mode='categorical',
                                                    batch_size=16,
                                                    shuffle=True,
                                                    seed=42)


Found 900 validated image filenames belonging to 4 classes.
Found 260 validated image filenames belonging to 4 classes.


# Model

## Custom Callbacks function


### Create new metrics for monitoring

In [7]:
from sklearn.metrics import f1_score, recall_score, precision_score
import numpy as np
import os


class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = np.argmax(self.model.predict(self.validation_data[0]), -1)
        val_targ = self.validation_data[1]
        if len(val_targ.shape) == 2 and val_targ.shape[1] != 1:
            val_targ = np.argmax(val_targ, -1)

        _val_f1 = f1_score(val_targ, val_predict, average='macro')
        _val_recall = recall_score(val_targ, val_predict, average='macro')
        _val_precision = precision_score(val_targ, val_predict, average='macro')

        logs['val_f1'] = _val_f1
        logs['val_recall'] = _val_recall
        logs['val_precision'] = _val_precision
        print(" — val_f1: %f — val_precision: %f — val_recall: %f" % (_val_f1, _val_precision, _val_recall))
        return

### Create f1-score metric

In [8]:
from keras import backend as K


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras


def specificity(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    return tn / (tn + fp + K.epsilon())


def negative_predictive_value(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))
    return tn / (tn + fn + K.epsilon())


def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))


def fbeta(y_true, y_pred, beta=2):
    y_pred = K.clip(y_pred, 0, 1)

    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    num = (1 + beta ** 2) * (p * r)
    den = (beta ** 2 * p + r + K.epsilon())
    return K.mean(num / den)


def matthews_correlation_coefficient(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())


def equal_error_rate(y_true, y_pred):
    n_imp = tf.count_nonzero(tf.equal(y_true, 0), dtype=tf.float32) + tf.constant(K.epsilon())
    n_gen = tf.count_nonzero(tf.equal(y_true, 1), dtype=tf.float32) + tf.constant(K.epsilon())

    scores_imp = tf.boolean_mask(y_pred, tf.equal(y_true, 0))
    scores_gen = tf.boolean_mask(y_pred, tf.equal(y_true, 1))

    loop_vars = (tf.constant(0.0), tf.constant(1.0), tf.constant(0.0))
    cond = lambda t, fpr, fnr: tf.greater_equal(fpr, fnr)
    body = lambda t, fpr, fnr: (
        t + 0.001,
        tf.divide(tf.count_nonzero(tf.greater_equal(scores_imp, t), dtype=tf.float32), n_imp),
        tf.divide(tf.count_nonzero(tf.less(scores_gen, t), dtype=tf.float32), n_gen)
    )
    t, fpr, fnr = tf.while_loop(cond, body, loop_vars, back_prop=False)
    eer = (fpr + fnr) / 2

    return eer

### Warmup and Cosine Learning rate eecay

In [ ]:
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import (
    Callback,
    LearningRateScheduler,
    TensorBoard
)


def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """Cosine decay schedule with warm up period.

    Cosine annealing learning rate as described in:
      Loshchilov and Hutter, SGDR: Stochastic Gradient Descent with Warm Restarts.
      ICLR 2017. https://arxiv.org/abs/1608.03983
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.

    Arguments:
        global_step {int} -- global step.
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
    Returns:
      a float representing learning rate.

    Raises:
      ValueError: if warmup_learning_rate is larger than learning_rate_base,
        or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(keras.callbacks.Callback):
    """Cosine decay with warmup learning rate scheduler
    """

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """Constructor for cosine decay with warmup learning rate scheduler.

    Arguments:
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        global_step_init {int} -- initial global step, e.g. from previous checkpoint.
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
        verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))

## Transfer learning with Xception model

### Create model

In [12]:
xception_model_input = tf.keras.applications.xception.Xception(weights='imagenet', include_top=False, input_shape=(224, 224,3), classes=4)

for layers in xception_model_input.layers[:125]:
    layers.trainable=False


xception_model = tf.keras.layers.Flatten()(xception_model_input.output)
xception_model = tf.keras.layers.Dropout(0.5)(xception_model)
xception_model = tf.keras.layers.Dense(4, activation='softmax')(xception_model)
xception_model_final = tf.keras.Model(inputs=xception_model_input.input, outputs=xception_model)

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32) # I use ._decayed_lr method instead of .lr
    return lr
# learning_rate = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=1e-3, first_decay_steps=6, t_mul=2.0, m_mul=1.0, alpha=1e-10, name=None)
optimizer = tf.keras.optimizers.Adam(lr=1e-4)
lr_metric = get_lr_metric(optimizer)
xception_model_final.compile(loss = 'categorical_crossentropy', optimizer= optimizer, metrics=['accuracy', f1, lr_metric])


In [ ]:
xception_model_final.summary()

### Create checkpoint callback

In [13]:
try:
  os.mkdir('checkpoint')
except:
  pass
xception_filepath = 'checkpoint/xception125-no-dense_batch16-do50-lr_const'+'.h5'
xception_checkpoint = tf.keras.callbacks.ModelCheckpoint(xception_filepath, monitor='val_f1', 
                                                 mode='max', verbose=1,
                                                 save_best_only=True,
                                                 save_weights_only=False)

In [ ]:
number_of_epochs = 200

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1', mode='max', factor=0.3, patience=3, min_lr=1e-7)

xception_callbacklist = [xception_checkpoint]
xception_history = xception_model_final.fit(train_data_multi, epochs = number_of_epochs ,validation_data = test_data_multi, callbacks=xception_callbacklist,verbose=1)

Epoch 1/200
57/57 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.7144 - f1: 0.6983 - lr: 1.0000e-04
Epoch 1: val_f1 improved from -inf to 0.80143, saving model to checkpoint/xception125-no-dense_batch16-do50-lr_const.h5
57/57 [==============================] - 47s 736ms/step - loss: 0.6929 - accuracy: 0.7144 - f1: 0.6983 - lr: 1.0000e-04 - val_loss: 0.4727 - val_accuracy: 0.8115 - val_f1: 0.8014 - val_lr: 1.0000e-04
Epoch 2/200
57/57 [==============================] - ETA: 0s - loss: 0.4008 - accuracy: 0.8533 - f1: 0.8614 - lr: 1.0000e-04
Epoch 2: val_f1 improved from 0.80143 to 0.88873, saving model to checkpoint/xception125-no-dense_batch16-do50-lr_const.h5
57/57 [==============================] - 41s 723ms/step - loss: 0.4008 - accuracy: 0.8533 - f1: 0.8614 - lr: 1.0000e-04 - val_loss: 0.3557 - val_accuracy: 0.8846 - val_f1: 0.8887 - val_lr: 1.0000e-04
Epoch 3/200
57/57 [==============================] - ETA: 0s - loss: 0.2858 - accuracy: 0.9011 - f1: 0.8934

## Transfer learning with VGG16

In [ ]:
vgg16_model_input = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max', input_shape=(224, 224,3), classes=4)

for layers in vgg16_model_input.layers:
    layers.trainable=False


vgg16_model = tf.keras.layers.Flatten()(vgg16_model_input.output)
# vgg16_model = tf.keras.layers.Dropout(0.2)(vgg16_model)
vgg16_model = tf.keras.layers.Dense(4, activation='softmax')(vgg16_model)
vgg16_model_final = tf.keras.Model(inputs=vgg16_model_input.input, outputs=vgg16_model)

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32) # I use ._decayed_lr method instead of .lr
    return lr
# learning_rate = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=1e-3, first_decay_steps=6, t_mul=2.0, m_mul=1.0, alpha=1e-10, name=None)
optimizer = tf.keras.optimizers.Adam(lr=5e-5, clipnorm=1.)
# lr_metric = get_lr_metric(optimizer)
vgg16_model_final.compile(loss = 'categorical_crossentropy', optimizer= optimizer, metrics=['accuracy', f1])


58900480/58889256 [==============================] - 0s 0us/step


## Training model

In [ ]:
number_of_epochs = 200
vgg16_filepath = 'checkpoint/vgg16'+'.h5'
vgg16_checkpoint = tf.keras.callbacks.ModelCheckpoint(vgg16_filepath, monitor='val_f1', 
                                                 mode='max', verbose=1,
                                                 save_best_only=True,
                                                 save_weights_only=False)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1', mode='max', factor=0.3, patience=3, min_lr=1e-7)

vgg16_callbacklist = [vgg16_checkpoint]
vgg16_history = vgg16_model_final.fit(train_data_multi, epochs = number_of_epochs ,validation_data = test_data_multi, callbacks=vgg16_callbacklist,verbose=1)


Epoch 1/200
57/57 [==============================] - ETA: 0s - loss: 1.7110 - accuracy: 0.3800 - f1: 0.3303
Epoch 1: val_f1 improved from -inf to 0.34765, saving model to checkpoint/vgg16.h5
57/57 [==============================] - 370s 6s/step - loss: 1.7110 - accuracy: 0.3800 - f1: 0.3303 - val_loss: 1.5421 - val_accuracy: 0.3846 - val_f1: 0.3477
Epoch 2/200
57/57 [==============================] - ETA: 0s - loss: 1.4070 - accuracy: 0.4033 - f1: 0.2876
Epoch 2: val_f1 did not improve from 0.34765
57/57 [==============================] - 43s 751ms/step - loss: 1.4070 - accuracy: 0.4033 - f1: 0.2876 - val_loss: 1.3466 - val_accuracy: 0.3846 - val_f1: 0.2517
Epoch 3/200
57/57 [==============================] - ETA: 0s - loss: 1.2711 - accuracy: 0.4222 - f1: 0.2785
Epoch 3: val_f1 did not improve from 0.34765
57/57 [==============================] - 43s 750ms/step - loss: 1.2711 - accuracy: 0.4222 - f1: 0.2785 - val_loss: 1.2294 - val_accuracy: 0.4462 - val_f1: 0.2797
Epoch 4/200
57/57 [

## Dislay Model training history

In [ ]:
print(xception_history.history.keys())

### Summarize history for f1-score

In [ ]:
plt.plot(xception_history.history['accuracy'])
plt.plot(xception_history.history['val_accuracy'])
plt.title('model f1')
plt.ylabel('val_f1')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Load checkpoint model for continue training

In [ ]:
loaded_model = tf.keras.models.load_model('checkpoint/vgg16.h5', custom_objects={"f1": f1 })
vgg16_checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoint/vgg16.h5', monitor='val_f1', 
                                                 mode='max', verbose=1,
                                                 save_best_only=True,
                                                 save_weights_only=False)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1', mode='max', factor=0.1, patience=3, min_lr=0.00000000001)
vgg16_callbacklist = [vgg16_checkpoint]
# retraining the model
loaded_model.fit(train_data_multi, epochs = 100 ,validation_data = test_data_multi,callbacks=vgg16_callbacklist,verbose=1)

Epoch 1/100
57/57 [==============================] - ETA: 0s - loss: 0.4756 - accuracy: 0.8278 - f1: 0.8060
Epoch 1: val_f1 improved from -inf to 0.75587, saving model to checkpoint/vgg16.h5
57/57 [==============================] - 337s 6s/step - loss: 0.4756 - accuracy: 0.8278 - f1: 0.8060 - val_loss: 0.5688 - val_accuracy: 0.7846 - val_f1: 0.7559
Epoch 2/100
57/57 [==============================] - ETA: 0s - loss: 0.4762 - accuracy: 0.8256 - f1: 0.8165
Epoch 2: val_f1 improved from 0.75587 to 0.77107, saving model to checkpoint/vgg16.h5
57/57 [==============================] - 41s 725ms/step - loss: 0.4762 - accuracy: 0.8256 - f1: 0.8165 - val_loss: 0.5370 - val_accuracy: 0.7923 - val_f1: 0.7711
Epoch 3/100
57/57 [==============================] - ETA: 0s - loss: 0.4509 - accuracy: 0.8356 - f1: 0.8299
Epoch 3: val_f1 improved from 0.77107 to 0.78583, saving model to checkpoint/vgg16.h5
57/57 [==============================] - 41s 718ms/step - loss: 0.4509 - accuracy: 0.8356 - f1: 0.8

### Predict with testing dataset

In [ ]:
predict_df = pd.read_csv("REMIS_Image_Dataset/test_re.csv")
predict_df['labels'] = [predict_df['path'].str.split('\\')[i][1] for i in range(len(predict_df['path']))]
predict_df['path'] = predict_df['path'].str.replace('Dataset', 'REMIS_Image_Dataset')
predict_df['path'] = predict_df['path'].str.replace('\\', '/')
predict_df['name'] = ['Test' + str(i) + '.jpg' for i in range(len(predict_df['path'])) ]
predict_df

test_data_gen = ImageDataGenerator(rescale=1/255.)

test_data= test_data_generator.flow_from_dataframe(dataframe=predict_df,
                                                    directory="data/testing",
                                                    x_col="name",
                                                    y_col=None,
                                                    target_size=(224, 224),
                                                    class_mode=None,
                                                    batch_size=16,
                                                    shuffle=False,
                                                    seed=42)
test_model = tf.keras.models.load_model('checkpoint/xception125-no-dense_batch16_flip-do20-reduce_lr-from-1e3.h5', custom_objects={"f1": f1, "lr": lr_metric })
test_data.reset()
pred=test_model.predict_generator(test_data, verbose=0)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_data_multi.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predict_df = predict_df.assign(predict = predictions)
display(predict_df)
true_count=0
false = []
for i in range(260):
    if predict_df['labels'][i] == predict_df['predict'][i]:
        true_count+=1
  # else:
  #   false.append(predict_df['predict'][i])
print(true_count, true_count/260, '\n')

Found 260 validated image filenames.


,path,label,labels,name,predict
0,REMIS_Image_Dataset/others/837645183007172.jpeg,"[0, 1, 0, 0]",others,Test0.jpg,others
1,REMIS_Image_Dataset/post/123nhadat_4321799_4.jpg,"[0, 0, 1, 0]",post,Test1.jpg,project
2,REMIS_Image_Dataset/others/10208474191744749.jpeg,"[0, 1, 0, 0]",others,Test2.jpg,others
3,REMIS_Image_Dataset/investor/rever_bat-dong-sa...,"[1, 0, 0, 0]",investor,Test3.jpg,investor
4,REMIS_Image_Dataset/project/duanbatdongsan_pj5...,"[0, 0, 0, 1]",project,Test4.jpg,project
...,...,...,...,...,...
255,REMIS_Image_Dataset/investor/rever_gia-tue_0.jpg,"[1, 0, 0, 0]",investor,Test255.jpg,investor
256,REMIS_Image_Dataset/post/123nhadat_4310483_0.jpg,"[0, 0, 1, 0]",post,Test256.jpg,post
257,REMIS_Image_Dataset/others/10200930046625836.jpeg,"[0, 1, 0, 0]",others,Test257.jpg,others
258,REMIS_Image_Dataset/others/1047558682015820.jpeg,"[0, 1, 0, 0]",others,Test258.jpg,others


233 0.8961538461538462 

